# 1. 课堂示例代码复刻

## 1.1 导入数据

In [63]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [64]:
df = pd.read_csv('text_analysis_weibo.csv', index_col = 0)

In [65]:
df.head()

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东


## 1.2 语料预处理 

### 1.2.1 剔除符号与数字

In [66]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心'

### 1.2.2 分词

In [67]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

'高校 通报 教师 图书馆 打电话 声音 过大 出言不逊 公道 人心 谣言 人心 \u200b \u200b'

In [68]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 公道, 人心, 谣..."
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,[转发]
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,"[刘雨昕, 运动, 濛, 河山, 覆, 冰雪, 健儿, 冬奥, 金牌, 全能, 唱, 跳, ..."
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,"[丁程鑫, 丁程鑫, 后援会, 官博丁, 程鑫, 超话, 丁程鑫, 二十, 成金筑梦鑫, 世..."
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,"[诶, 不要脸, 诶, 资格, 宣传, 奥运, 抵制, 抵制]"
...,...,...,...,...,...,...,...,...,...
49995,935天气路况早知道 2022年3月2日 星期三 今日限行尾号3和8 8：00分发布 ...,0,0,0,2231ab79796d569625633acafd9f1351,445,74,河南,"[天气, 路况, 早, 年月日, 星期三, 今日, 限行, 尾号, 发布, 天气, 月, 日..."
49996,4月1日起，交通违法记分规则重大调整！ http://t.cn/A66zxfR0 ​​,0,0,0,91c9d380c1cc7811be5ee10ce510d77a,842,54,甘肃,"[月, 日, 交通, 违法, 记分, 规则, 调整]"
49997,转发C,0,0,1,301771db13798e2dd493c02a44e59444,99,3433,江苏,[转发]
49998,全体 甘肃车主注意 4月1日起，全面实施！ ...,0,0,0,NaN,0,0,甘肃,"[全体, 甘肃, 车主, 月, 日, 全面实施, 记分, 分值, 调整, 为分, 分分, 分..."


In [69]:
#数据太大，把数据换小一点
df = df.iloc[:2000]
df

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 公道, 人心, 谣..."
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,[转发]
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,"[刘雨昕, 运动, 濛, 河山, 覆, 冰雪, 健儿, 冬奥, 金牌, 全能, 唱, 跳, ..."
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,"[丁程鑫, 丁程鑫, 后援会, 官博丁, 程鑫, 超话, 丁程鑫, 二十, 成金筑梦鑫, 世..."
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,"[诶, 不要脸, 诶, 资格, 宣传, 奥运, 抵制, 抵制]"
...,...,...,...,...,...,...,...,...,...
1995,#高校通报教师图书馆打电话声音过大出言不逊#恼羞成怒，口不择言了呗。其实就是觉得自己是老师，...,0,1,0,e443523e56e815a9f4cb6a5e8c39e9ac,0,120,福建,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 恼羞成怒, 口不择..."
1996,#高校通报教师图书馆打电话声音过大出言不逊#BWL给我学[怒] ​​,0,0,0,806b37be1bdda2a9de2b18e4f6bd6eae,173,190,陕西,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 学怒]"
1997,#高校通报教师图书馆打电话声音过大出言不逊# 希望冲到热一捏 ​​,0,0,0,1308a7235b622ac5da23bc8905bc4dff,234,422,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 希望, 冲到, 热..."
1998,#高校通报教师图书馆打电话声音过大出言不逊#“我告你性骚扰”[费解][费解][费解] 胡搅蛮...,0,0,0,2d3ff42567d02e04adf399c7d966ff7c,10,657,福建,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 我告, 性骚扰, ..."


## 1.3 LDA

### 1.3.1 构建模型

In [70]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [71]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [72]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.028*"刘雨昕" + 0.026*"运动" + 0.024*"冬奥" + 0.024*"濛" + 0.016*"北京"')
(1, '0.058*"图书馆" + 0.054*"教师" + 0.053*"高校" + 0.052*"打电话" + 0.051*"声音"')
(2, '0.009*"哈哈哈" + 0.007*"苦涩" + 0.006*"学生" + 0.006*"真源" + 0.006*"定律"')
(3, '0.019*"冬奥" + 0.018*"北京" + 0.016*"冬奥会" + 0.011*"未来" + 0.010*"加油"')
(4, '0.019*"肖战" + 0.013*"少年" + 0.011*"抱" + 0.011*"一抱" + 0.007*"代言人"')


In [73]:
df['微博内容分词'].iloc[0]

['高校', '通报', '教师', '图书馆', '打电话', '声音', '过大', '出言不逊', '公道', '人心', '谣言', '人心']

In [74]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9383360743522644	 
Topic: 0.058*"图书馆" + 0.054*"教师" + 0.053*"高校" + 0.052*"打电话" + 0.051*"声音" + 0.051*"出言不逊" + 0.050*"通报" + 0.050*"过大" + 0.027*"转发" + 0.020*"老师"

Score: 0.015472513623535633	 
Topic: 0.019*"冬奥" + 0.018*"北京" + 0.016*"冬奥会" + 0.011*"未来" + 0.010*"加油" + 0.008*"中国" + 0.007*"冰雪" + 0.007*"周深" + 0.006*"墩" + 0.005*"冰墩"

Score: 0.015398167073726654	 
Topic: 0.019*"肖战" + 0.013*"少年" + 0.011*"抱" + 0.011*"一抱" + 0.007*"代言人" + 0.007*"时代" + 0.007*"超话" + 0.007*"北京" + 0.005*"品牌" + 0.005*"宋亚轩"

Score: 0.01539703831076622	 
Topic: 0.009*"哈哈哈" + 0.007*"苦涩" + 0.006*"学生" + 0.006*"真源" + 0.006*"定律" + 0.006*"老师" + 0.005*"北京" + 0.005*"刘彰" + 0.005*"用户" + 0.004*"中国"

Score: 0.015396206639707088	 
Topic: 0.028*"刘雨昕" + 0.026*"运动" + 0.024*"冬奥" + 0.024*"濛" + 0.016*"北京" + 0.009*"加油" + 0.008*"期待" + 0.007*"全能" + 0.006*"冬奥会" + 0.006*"健儿"


In [75]:
documents = df['微博内容分词'].values

In [76]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.015396207), (1, 0.93833613), (2, 0.015397038), (3, 0.015472431), (4, 0.015398167)]

Document 2:
[(0, 0.10006928), (1, 0.5999143), (2, 0.10000268), (3, 0.100013115), (4, 0.10000065)]

Document 3:
[(0, 0.9702997)]

Document 4:
[(0, 0.17901462), (4, 0.7977339)]

Document 5:
[(0, 0.022233428), (1, 0.022421192), (2, 0.022307403), (3, 0.022896357), (4, 0.91014165)]

Document 6:
[(0, 0.10006906), (1, 0.59991455), (2, 0.10000268), (3, 0.100013115), (4, 0.10000065)]

Document 7:
[(0, 0.14274558), (2, 0.83692026)]

Document 8:
[(0, 0.100068964), (1, 0.5999146), (2, 0.100002676), (3, 0.1000131), (4, 0.10000064)]

Document 9:
[(0, 0.012018921), (1, 0.011792572), (2, 0.011841239), (3, 0.95251596), (4, 0.011831311)]

Document 10:
[(0, 0.033739135), (1, 0.03333832), (2, 0.033342324), (3, 0.86624056), (4, 0.033339612)]



### 1.3.2 可视化

In [77]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis,local=True)

In [78]:
# 导出可视化结果到html
pyLDAvis.save_html(lda_vis, 'example.html')

# 2 自我探索代码展示

## 2.1 导入数据

In [79]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [80]:
#读取数据
df = pd.read_csv('combined_data.csv', index_col = 0)

In [81]:
df.head()

,title,content
date,,
20160204,传承红色基因 让人民生活蒸蒸日上,春佳节前夕，习近平总书记来到了江西，深入乡村、企业、学校、社区、革命根据地纪念馆，看望慰问广...
20160204,李克强主持召开国务院常务会议,国务院总理李克强2月3日主持召开国务院常务会议，部署建设双创基地发展众创空间，加快培育新动能...
20160204,俞正声与全国性宗教团体负责人迎春座谈,中共中央政治局常委、全国政协主席俞正声4日邀请全国性宗教团体负责人座谈，共庆新春佳节，并代表...
20160204,俞正声会见柬埔寨副首相,全国政协主席俞正声4日在人民大会堂会见柬埔寨副首相兼外交国际合作大臣贺南洪。 俞...
20160204,国务院印发《关于钢铁行业化解过剩产能实现脱困发展的意见》,经李克强总理签批，国务院日前印发《关于钢铁行业化解过剩产能实现脱困发展的意见》。 ...


## 2.2 语料预处理

### 2.2.1 剔除符号和数字

In [82]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['content'][0]
remove_nums(test)[:100]

C:\Users\Sun Yedi\AppData\Local\Temp\ipykernel_9796\1591612704.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test = df['content'][0]


'春佳节前夕习近平总书记来到了江西深入乡村企业学校社区革命根据地纪念馆看望慰问广大干部群众为他们送去党中央的新春祝福和亲切关怀总书记的殷切期望和真挚话语让广大干部群众倍感温暖革命老区井冈山是习总书记这次'

### 2.2.2 分词

In [83]:
#加载中文停用词词典
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['content'][0]
clean_text(test)

C:\Users\Sun Yedi\AppData\Local\Temp\ipykernel_9796\2982172304.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test = df['content'][0]


'春 佳节 前夕 习近平 总书记 来到 江西 乡村 企业 学校 社区 革命 根据地 纪念馆 看望 慰问 广大干部 群众 送 党中央 新春 祝福 亲切 关怀 总书记 殷切期望 真挚 话语 广大干部 群众 倍感 温暖 \u3000 \u3000 革命 老区 井冈山 是习 总书记 这次 考察 调研 第一站 八角楼 革命 旧址 群 总书记 革命烈士 后代 先进人物 代表 围炉 坐 \u3000 \u3000 井冈山 中国 革命 摇篮 总书记 指出 井冈山 精神 放射出 时代 光芒 87 岁 全国 道德 模范 毛秉华 义务 宣讲 井冈山 精神 至今 万多 场 听众 220 万人次 回忆起 总书记 交谈 毛老 总书记 叮嘱 宣讲 \u3000 \u3000 扶贫 路上 落下 一个 贫困家庭 丢下 一个 贫困 群众 总书记 话语 茅坪 乡 干部群众 深受鼓舞 设在 神 山村 村口 精准 扶贫 办公室 总书记 来到 村里 第一个 视察 地方 \u3000 \u3000 井冈山市 茅坪 乡 党委书记 兰胜华 没想到 摆 资料 随便 翻一翻 总书记 仔细 一看 东西 门道 一个 老人 带 两个 孙女 儿子 他家 变故 确实 感觉 总书记 心系 群众 心系 人民 革命 老区 精准 扶贫 工作 看得 很重 感觉 工作 总书记 学习 真的 细 细 一个 大国 总书记 工作 关注 理由 做好 工作 \u3000 \u3000 井冈山市 茅坪 乡神 山村 村民 彭 水生 领导 远 穷山沟 穷山沟 福气 中国 人民 福气 不错 嘞 书记 群众 中央 政策 老百姓 关心 \u3000 \u3000 这是 总书记 离开 神 山村 场景 村民 围拢 久久 不愿 散 \u3000 \u3000 彭 水生 村里 老支书 身体 原因 贫困户 四年 扶贫办 领了 8 头 羊羔 养到 19 头 家里 3 亩 黄桃 挂果 眼看 就要 摘掉 贫困 帽子 听 总书记 支持 老区 发展 盼头 \u3000 \u3000 东湖区 彭家桥 街道 光明 社区 社区 合唱团 送给 总书记 春节 礼物 填词 南泥湾 新唱 首歌 创作 党 十八 之际 学 首歌 想到 唱 总书记 听 \u3000 \u3000 南昌市 东湖区 彭家桥 街道 光明 社区 居民 郭晓琳 我话 流眼泪 幸福 眼泪 南泥湾 新唱 唱 十三亿 中国 托起 中国

In [84]:
df['content'] = df['content'].astype(str)
df['content分词'] = df['content'].apply(remove_nums)
df['content分词'] = df['content分词'].apply(clean_text)
df['content分词'] = df['content分词'].apply(lambda x: x.split())
df[:100]

,title,content,content分词
date,,,
20160204,传承红色基因 让人民生活蒸蒸日上,春佳节前夕，习近平总书记来到了江西，深入乡村、企业、学校、社区、革命根据地纪念馆，看望慰问广...,"[春, 佳节, 前夕, 习近平, 总书记, 来到, 江西, 乡村, 企业, 学校, 社区, ..."
20160204,李克强主持召开国务院常务会议,国务院总理李克强2月3日主持召开国务院常务会议，部署建设双创基地发展众创空间，加快培育新动能...,"[国务院, 总理, 李克强, 月, 日, 主持, 国务院, 常务会议, 部署, 建设, 双创..."
20160204,俞正声与全国性宗教团体负责人迎春座谈,中共中央政治局常委、全国政协主席俞正声4日邀请全国性宗教团体负责人座谈，共庆新春佳节，并代表...,"[中共, 中央政治局常委, 全国政协, 主席, 俞正声, 日, 邀请, 全国性, 宗教团体,..."
20160204,俞正声会见柬埔寨副首相,全国政协主席俞正声4日在人民大会堂会见柬埔寨副首相兼外交国际合作大臣贺南洪。 俞...,"[全国政协, 主席, 俞正声, 日, 人民大会堂, 会见, 柬埔寨, 副, 首相, 兼, 外..."
20160204,国务院印发《关于钢铁行业化解过剩产能实现脱困发展的意见》,经李克强总理签批，国务院日前印发《关于钢铁行业化解过剩产能实现脱困发展的意见》。 ...,"[李克强, 总理, 签批, 国务院, 日前, 印发, 钢铁行业, 化解, 过剩, 产能, 脱..."
...,...,...,...
20160212,瞰春：贵州石阡 毛龙舞出绿色与和谐,仡佬族是主要聚居在贵州的一个古老的少数民族，春节他们有舞毛龙的习俗，今天的《瞰春》，让我们去...,"[仡佬族, 聚居, 贵州, 一个, 古老, 少数民族, 春节, 有舞, 毛龙, 习俗, 瞰,..."
20160212,【新春走基层】飞翔在深山雪海中的千纸鹤,吉林省军区边防某团维东哨所座落在长白山麓，春节期间，记者走近这个雪海孤岛，目睹了官兵戍守边关...,"[吉林省, 军区, 边防, 某团, 维东, 哨所, 座落在, 长白, 山麓, 春节, 期间,..."
20160212,【新春走基层·孝顺怎么做】始于事亲 终于报国,有很多人因为工作繁忙，陪伴父母长辈的时间比较少。在陪伴和工作之间，父母、长辈怎么看待孝顺？子...,"[很多, 工作, 繁忙, 陪伴, 父母, 长辈, 时间, 陪伴, 工作, 父母, 长辈, 看..."


## 2.3 LDA

### 2.3.1 构建模型

In [85]:
dictionary = corpora.Dictionary(df['content分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['content分词']]     # 根据分词结果创建语料库

In [86]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [87]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.027*"日" + 0.024*"美国" + 0.018*"病例" + 0.014*"月" + 0.010*"确诊"')
(1, '0.029*"中国" + 0.023*"发展" + 0.023*"合作" + 0.015*"习近平" + 0.011*"中方"')
(2, '0.020*"发展" + 0.009*"建设" + 0.008*"企业" + 0.008*"经济" + 0.008*"工作"')
(3, '0.015*"月" + 0.011*"日" + 0.007*"我国" + 0.006*"中国" + 0.006*"全国"')
(4, '0.018*"习近平" + 0.012*"党" + 0.011*"中国" + 0.010*"工作" + 0.010*"时代"')


In [88]:
df['content分词'].iloc[0]

['春',
 '佳节',
 '前夕',
 '习近平',
 '总书记',
 '来到',
 '江西',
 '乡村',
 '企业',
 '学校',
 '社区',
 '革命',
 '根据地',
 '纪念馆',
 '看望',
 '慰问',
 '广大干部',
 '群众',
 '送',
 '党中央',
 '新春',
 '祝福',
 '亲切',
 '关怀',
 '总书记',
 '殷切期望',
 '真挚',
 '话语',
 '广大干部',
 '群众',
 '倍感',
 '温暖',
 '革命',
 '老区',
 '井冈山',
 '是习',
 '总书记',
 '这次',
 '考察',
 '调研',
 '第一站',
 '八角楼',
 '革命',
 '旧址',
 '群',
 '总书记',
 '革命烈士',
 '后代',
 '先进人物',
 '代表',
 '围炉',
 '坐',
 '井冈山',
 '中国',
 '革命',
 '摇篮',
 '总书记',
 '指出',
 '井冈山',
 '精神',
 '放射出',
 '时代',
 '光芒',
 '岁',
 '全国',
 '道德',
 '模范',
 '毛秉华',
 '义务',
 '宣讲',
 '井冈山',
 '精神',
 '至今',
 '万多',
 '场',
 '听众',
 '万人次',
 '回忆起',
 '总书记',
 '交谈',
 '毛老',
 '总书记',
 '叮嘱',
 '宣讲',
 '扶贫',
 '路上',
 '落下',
 '一个',
 '贫困家庭',
 '丢下',
 '一个',
 '贫困',
 '群众',
 '总书记',
 '话语',
 '茅坪',
 '乡',
 '干部群众',
 '深受鼓舞',
 '设在',
 '神',
 '山村',
 '村口',
 '精准',
 '扶贫',
 '办公室',
 '总书记',
 '来到',
 '村里',
 '第一个',
 '视察',
 '地方',
 '井冈山市',
 '茅坪',
 '乡',
 '党委书记',
 '兰胜华',
 '没想到',
 '摆',
 '资料',
 '随便',
 '翻一翻',
 '总书记',
 '仔细',
 '一看',
 '东西',
 '门道',
 '一个',
 '老人',
 '带',
 '两个',
 '孙女',
 '儿子',
 '他家',
 '变故',
 '确实',
 '感觉',
 '总

In [89]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.48872146010398865	 
Topic: 0.018*"习近平" + 0.012*"党" + 0.011*"中国" + 0.010*"工作" + 0.010*"时代" + 0.010*"人民" + 0.009*"总书记" + 0.009*"社会主义" + 0.009*"会议" + 0.007*"发展"

Score: 0.34842973947525024	 
Topic: 0.015*"月" + 0.011*"日" + 0.007*"我国" + 0.006*"中国" + 0.006*"全国" + 0.005*"增长" + 0.004*"北京" + 0.004*"活动" + 0.004*"同比" + 0.003*"亿元"

Score: 0.16133010387420654	 
Topic: 0.020*"发展" + 0.009*"建设" + 0.008*"企业" + 0.008*"经济" + 0.008*"工作" + 0.007*"创新" + 0.007*"推进" + 0.006*"推动" + 0.006*"疫情" + 0.006*"国家"


In [90]:
documents = df['content分词'].values

In [91]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(2, 0.16132945), (3, 0.34843025), (4, 0.4887216)]

Document 2:
[(2, 0.9094683), (3, 0.08919553)]

Document 3:
[(1, 0.0827039), (2, 0.34681293), (4, 0.56901485)]

Document 4:
[(1, 0.9495268), (4, 0.043890752)]

Document 5:
[(2, 0.91553485), (3, 0.07334488)]

Document 6:
[(2, 0.20757735), (3, 0.030196784), (4, 0.75596154)]

Document 7:
[(1, 0.12774235), (4, 0.86141783)]

Document 8:
[(2, 0.052501094), (3, 0.43731576), (4, 0.5004693)]

Document 9:
[(3, 0.9922475)]

Document 10:
[(0, 0.034410052), (3, 0.9596855)]



### 2.3.2 可视化

In [92]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis,local=True)

In [93]:
# 导出可视化结果到html
pyLDAvis.save_html(lda_vis, 'mytest.html')